In [4]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

In [2]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, 
                                names=['tuned_dlm_rm3_unjudged'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, 
                    names=['tuned_dlm_rm3_unjudged21'],
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, 
                names=['tuned_dlm_rm3_unjudged20'],
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.559156
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f1a29c23ef0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbc7c7758 at 0x7f1a29db8430>>) fb_terms=6', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f1a29c23ef0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbc7c7758 at 0x7f1a29db8430>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f1a29c23ef0 jclass=org/terrier/querying/RM3 jself=<LocalRef

/tmp/ipykernel_2391585/3209819995.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels,
/tmp/ipykernel_2391585/3209819995.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels,
/tmp/ipykernel_2391585/3209819995.py:38: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of 

In [3]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.518446
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f1a29c10d60 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbd4845d0 at 0x7f19ddba7790>>) fb_terms=12', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f1a29c10d60 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbd4845d0 at 0x7f19ddba7790>>) fb_docs=6', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f1a29c10d60 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x

/tmp/ipykernel_2391585/3050515307.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391585/3050515307.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_r

\begin{tabular}{llrrr}
\toprule
{} &                                name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_dlm\_rm3\_unjudged &    0.726258 &     0.684178 &  0.711345 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_dlm\_rm3\_unjudged20 &    0.841113 &     0.773703 &  0.715838 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_dlm\_rm3\_unjudged21 &     0.67633 &      0.67509 &  0.706942 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
Fold 1
Best ndcg_cut_5 is 0.518249
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz

/tmp/ipykernel_2391585/3050515307.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391585/3050515307.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_r

\begin{tabular}{llrrr}
\toprule
{} &                                name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_dlm\_rm3\_unjudged &    0.721387 &     0.706998 &  0.723074 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_dlm\_rm3\_unjudged20 &    0.850653 &     0.810339 &  0.743247 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_dlm\_rm3\_unjudged21 &    0.655852 &     0.673461 &  0.703304 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/topicspd_index
Fold 1
Best ndcg_cut_5 is 0.514084
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/D

...

/tmp/ipykernel_2391585/3050515307.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391585/3050515307.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_r

\begin{tabular}{llrrr}
\toprule
{} &                                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_dlm\_rm3\_unjudged &    0.716561 &     0.693828 &  0.708333 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_dlm\_rm3\_unjudged20 &    0.846066 &     0.780281 &  0.705638 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_dlm\_rm3\_unjudged21 &    0.661343 &     0.684045 &  0.710974 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusions_leakage/pd_index
Fold 1
Best ndcg_cut_5 is 0.499859
Best setting is ['Quer

/tmp/ipykernel_2391585/3050515307.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391585/3050515307.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_r

\begin{tabular}{llrrr}
\toprule
{} &                                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_dlm\_rm3\_unjudged &    0.735142 &     0.686443 &  0.700421 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_dlm\_rm3\_unjudged20 &    0.861341 &     0.774573 &  0.710512 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_dlm\_rm3\_unjudged21 &    0.676654 &     0.684525 &  0.690531 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.509156
Best setting is ['QueryExpansion(/mnt/ceph/storage/d

/tmp/ipykernel_2391585/3050515307.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391585/3050515307.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_r

In [5]:
import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            i += 1
            


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.411759
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f19dd60e7c0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbd484530 at 0x7f19dd5fdf10>>) fb_terms=4', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f19dd60e7c0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbd484530 at 0x7f19dd5fdf10>>) fb_docs=6', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f19dd60e7c0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x555fbd484530 at 0x7f19dd5fdf10>>) fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.258482
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f19dd60e7c0 jclas

/tmp/ipykernel_2391585/4256079596.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391585/4256079596.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_r